# BPE Token Sequence List를 활용한 LSTM+CNN 기반 악성코드 탐지 모델 Test
- accuracy: 0.9912482726853984
- BPE Token을 생성하면서 같이 제작된 BPE Token list를 데이터로 사용했습니다.
- 해당 모델 구조를 학습시키 위해서는 pefile dataset(dikedataset을 추천합니다.)을 utils.py의 함수들을 활용하여 전처리를 한 뒤 사용하셔야 합니다.
- 전처리를 하는 방식에 대해서는 utils 디렉토리에 있는 test.ipynb 파일을 통해 확인하실 수 있습니다.

# 필요 모듈 설치

In [2]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# utils.py 파일이 있는 디렉토리로 위치 이동

In [4]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 패키지 로드

In [5]:
from utils import *

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.layers import Conv1D, MaxPooling1D, Dropout, Activation
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.utils.np_utils import to_categorical

import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data load

# opcode list load

In [8]:
filePath = '/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/files/opcode database/opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)

opcodes2 = []
for i in range(len(opcodes)):
    opcodes2.append([opcodes[i]])

print(opcodes)

['aaa', 'aad', 'aam', 'aas', 'adc', 'add', 'and', 'call', 'cbw', 'clc', 'cld', 'cli', 'cmc', 'cmp', 'cmpsb', 'cmpsw', 'cwd', 'daa', 'das', 'dec', 'div', 'esc', 'hlt', 'idiv', 'imul', 'in', 'inc', 'int', 'into', 'iret', 'jcc', 'ja', 'jae', 'jb', 'jbe', 'jc', 'je', 'jg', 'jge', 'jl', 'jle', 'jnle', 'jno', 'jnp', 'jns', 'jnz', 'jo', 'jp', 'jpe', 'jpo', 'js', 'jz', 'jcxz', 'jmp', 'lahf', 'lds', 'lea', 'les', 'lock', 'lodsb', 'lodsw', 'loop', 'loope', 'loopne', 'loopnz', 'loopz', 'loopx', 'mov', 'movsb', 'movsw', 'mul', 'neg', 'nop', 'not', 'or', 'out', 'pop', 'popf', 'push', 'pushf', 'rcl', 'rcr', 'repxx', 'rep', 'repe', 'repne', 'repnz', 'repz', 'ret', 'retn', 'retf', 'rol', 'ror', 'sahf', 'sal', 'sar', 'sbb', 'scasb', 'scasw', 'shl', 'shr', 'stc', 'std', 'sti', 'stosb', 'stosw', 'sub', 'test', 'wait', 'xchg', 'xlat', 'xor']


## vocab load

In [10]:
with open('/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/BPE 알고리즘2/2000vocab.p', 'rb') as file:
    vocab = pickle.load(file)

vocab = vocab + opcodes2

print(vocab)

[['add', 'add'], ['mov', 'mov'], ['add', 'add', 'add', 'add'], ['push', 'push'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['push', 'call'], ['push', 'mov'], ['dec', 'mov'], ['mov', 'mov', 'mov', 'mov'], ['pop', 'pop'], ['mov', 'cmp'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['test', 'je'], ['mov', 'mov', 'mov'], ['mov', 'jmp'], ['cmp', 'je'], ['mov', 'add'], ['nop', 'nop'], ['pop', 'ret'], ['push', 'push', 'call'], ['push', 'lea'], ['mov', 'xor'], ['mov', 'call'], ['push', 'call', 'add'], ['inc', 'add'], ['dec', 'mov', 'dec', 'mov'], ['dec', 'lea'], ['pop', 'mov'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['mov', 'inc'], ['mov', 'lea'], ['push', 'push', 'push', 'push'], ['push', 'mov', 'call'], ['mov', 'push'], ['a

In [11]:
vocab_number = [len(vocab) - i for i in range(len(vocab))]

print(vocab_number)

[2112, 2111, 2110, 2109, 2108, 2107, 2106, 2105, 2104, 2103, 2102, 2101, 2100, 2099, 2098, 2097, 2096, 2095, 2094, 2093, 2092, 2091, 2090, 2089, 2088, 2087, 2086, 2085, 2084, 2083, 2082, 2081, 2080, 2079, 2078, 2077, 2076, 2075, 2074, 2073, 2072, 2071, 2070, 2069, 2068, 2067, 2066, 2065, 2064, 2063, 2062, 2061, 2060, 2059, 2058, 2057, 2056, 2055, 2054, 2053, 2052, 2051, 2050, 2049, 2048, 2047, 2046, 2045, 2044, 2043, 2042, 2041, 2040, 2039, 2038, 2037, 2036, 2035, 2034, 2033, 2032, 2031, 2030, 2029, 2028, 2027, 2026, 2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 194

# 모델 가중치 Load

In [19]:
model= Sequential()
model.add(Embedding(len(vocab), 3000))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding = 'valid', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(48, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 3000)        6336000   
                                                                 
 dropout_1 (Dropout)         (None, None, 3000)        0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          960064    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 55)                26400     
                                                                 
 dense_2 (Dense)             (None, 48)                2688      
                                                      

# Sample file Test
- Test를 위해 utils 폴더 내부의 utils.py 파일을 활용한다.

## MalwareDetectionFunction을 통한 Malware 탐지
- 양성 샘플 파일이 양성으로 결과가 나오는지 확인
- return 1: 악성코드, return 0: 양성코드

In [20]:
# 모델을 저장할 경로를 설정해주시면 됩니다.
filename = 'BPESequenceBasedLSTMCNNMalwareDetectionModel.h5'

model.load_weights(filename)

In [21]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunction(model, fileName):
    try:
        #parse exe file
        exe = pefile.PE(fileName)
        #call the function we created earlier
        sampleOpcodeList = ExtractPefileOpcodes(opcodes, exe)
    except:
        print('Error File')
    
    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    sampleTokenIdList = TokenIdMapping(vocab, sampleTokenList)

    sampleTokenIdList = np.array([sampleTokenIdList])
    sampleTokenIdList = sequence.pad_sequences(sampleTokenIdList, maxlen=3000)

    y_pred = model.predict(sampleTokenIdList)
    Y_pred = np.argmax(y_pred,axis=1)

    return Y_pred[0]

- 결과가 0으로 양성파일을 탐지한 것을 확인할 수 있다.

In [22]:
fileName = '3f3fe9ecad7f30fc80cdfb678d7ca27a30d0575a73818746e98be9170d3be348.exe'

MalwareDetectionFunction(model, fileName)

0

# opcode Sequence List pickle을 입력으로 사용하는 탐지 함수
- 악성코드 파일을 직접 Google Drive 및 GitHub에 올릴 수 없기 때문에 pickle로 먼저 opcodeSequence를 추출하여 해당 파일을 바탕으로 악성코드를 탐지하는 함수를 제작하였다.

In [24]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunctionUsingPickle(model, pickleName):
    with open(pickleName, 'rb') as lf:
        sampleOpcodeList = pickle.load(lf)

    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    sampleTokenIdList = TokenIdMapping(vocab, sampleTokenList)

    sampleTokenIdList = np.array([sampleTokenIdList])
    sampleTokenIdList = sequence.pad_sequences(sampleTokenIdList, maxlen=3000)

    y_pred = model.predict(sampleTokenIdList)
    Y_pred = np.argmax(y_pred,axis=1)

    return Y_pred[0]

- 악성코드 파일이기 때문에 모델이 1로 분류하는 것을 확인할 수 있다.

In [25]:
pickleName = 'MalwareSample.p'

MalwareDetectionFunctionUsingPickle(model, pickleName)

1